Imports

In [1]:
import sqlite3
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import apriori

Gather data

In [2]:
def getDbAsDF(conn: sqlite3.Connection, sql: str):
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    columns = [column[0] for column in cursor.description]

    formatted_data = {}

    for i in range(len(columns)):
        dataList = []
        for j in data:
            dataList.append(j[i])
        formatted_data[columns[i]] = dataList

    df = pd.DataFrame(data=formatted_data, columns=columns)
    return df


# Connect to SQLite database
connection = sqlite3.connect(r'..\..\Data\merged.sqlite')

SQL = r"""
SELECT *
FROM order_details as OD
INNER JOIN order_header as OH ON OH.ORDER_NUMBER = OD.ORDER_NUMBER
INNER JOIN product as P ON P.PRODUCT_NUMBER = OD.PRODUCT_NUMBER
"""

Dataframe = getDbAsDF(connection, SQL)
print(Dataframe.columns)
Dataframe

Index(['ORDER_DETAIL_CODE', 'ORDER_NUMBER', 'PRODUCT_NUMBER', 'QUANTITY',
       'UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'ORDER_NUMBER',
       'RETAILER_NAME', 'RETAILER_SITE_CODE', 'RETAILER_CONTACT_CODE',
       'SALES_STAFF_CODE', 'SALES_BRANCH_CODE', 'ORDER_DATE',
       'ORDER_METHOD_CODE', 'PRODUCT_NUMBER', 'INTRODUCTION_DATE',
       'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN', 'PRODUCT_IMAGE',
       'LANGUAGE', 'PRODUCT_NAME', 'DESCRIPTION'],
      dtype='object')


,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,...,ORDER_METHOD_CODE,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,48616,1258,17,92,60.29,84.41,78.37,1258,VIP Department Stores,13,...,7,17,5-3-2013,3,60,.28,P17CE1SB3.jpg,EN,Hibernator Lite,The Hibernator Lite is the perfect summer slee...
1,48617,1352,17,190,60.29,84.41,67.08,1352,Falcon Outfitters,17,...,2,17,5-3-2013,3,60,.28,P17CE1SB3.jpg,EN,Hibernator Lite,The Hibernator Lite is the perfect summer slee...
2,48618,1544,17,82,60.29,84.41,78.37,1544,VIP Department Stores,13,...,2,17,5-3-2013,3,60,.28,P17CE1SB3.jpg,EN,Hibernator Lite,The Hibernator Lite is the perfect summer slee...
3,48619,1249,17,190,60.29,84.41,72.26,1249,Ultra Sports,4,...,5,17,5-3-2013,3,60,.28,P17CE1SB3.jpg,EN,Hibernator Lite,The Hibernator Lite is the perfect summer slee...
4,48620,1268,17,152,60.29,84.41,78.05,1268,Falcon Outfitters,19,...,7,17,5-3-2013,3,60,.28,P17CE1SB3.jpg,EN,Hibernator Lite,The Hibernator Lite is the perfect summer slee...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,102951,8614,11,16,250,350,325,8614,Sport Jonas,267,...,4,11,15-2-2011,2,250,.28,P11CE1TN2.jpg,EN,Star Lite,"A perfect tent for biking and hiking trips, co..."
43059,102952,8611,11,18,250,350,350,8611,Sport Jonas,265,...,5,11,15-2-2011,2,250,.28,P11CE1TN2.jpg,EN,Star Lite,"A perfect tent for biking and hiking trips, co..."
43060,102953,8591,11,18,250,350,325,8591,Campingspecialisten,261,...,7,11,15-2-2011,2,250,.28,P11CE1TN2.jpg,EN,Star Lite,"A perfect tent for biking and hiking trips, co..."
43061,102954,8619,11,18,250,350,350,8619,Sport Jonas,268,...,5,11,15-2-2011,2,250,.28,P11CE1TN2.jpg,EN,Star Lite,"A perfect tent for biking and hiking trips, co..."


Select & Format data

In [3]:
Data = Dataframe[["ORDER_NUMBER","PRODUCT_NAME"]]
Data = Data.T.drop_duplicates().T
Data.dropna(inplace = True)
OrderNrs = Data["ORDER_NUMBER"].unique()
Data

,ORDER_NUMBER,PRODUCT_NAME
0,1258,Hibernator Lite
1,1352,Hibernator Lite
2,1544,Hibernator Lite
3,1249,Hibernator Lite
4,1268,Hibernator Lite
...,...,...
43058,8614,Star Lite
43059,8611,Star Lite
43060,8591,Star Lite
43061,8619,Star Lite


In [4]:
dataset = []
for number in OrderNrs:
    products = []
    FoundRows = Data.loc[Data['ORDER_NUMBER'] == number]
    for row in FoundRows["PRODUCT_NAME"]:
        products.append(row)
    dataset.append(products)
print(dataset)

[['Hibernator Lite', 'Polar Ice', 'Polar Wave', 'Seeker 35', 'Star Dome'], ['Hibernator Lite', 'Canyon Mule Extreme Backpack', 'Firefly 4', 'Firefly Extreme', 'EverGlow Butane', 'Husky Rope 50', 'Husky Rope 200', 'Granite Climbing Helmet', 'Granite Carabiner', 'Firefly Charger', 'Granite Chalk Bag', 'Granite Grip', 'Granite Axe', 'Mountain Man Extreme', 'BugShield Lotion', 'TrailChef Water Bag', 'Star Dome', 'Star Gazer 6'], ['Hibernator Lite', 'Hibernator', 'Hibernator Pad', 'Firefly 2'], ['Hibernator Lite', 'Canyon Mule Journey Backpack', 'Firefly Extreme', 'Calamine Relief', 'TrailChef Cup', 'TrailChef Utensils'], ['Hibernator Lite', 'Hibernator Pillow', 'Husky Harness Extreme', 'Granite Belay', 'Firefly Climbing Lamp', 'Firefly Charger', 'Firefly Rechargeable Battery', 'Granite Axe', 'Granite Extreme', 'Mountain Man Extreme', 'Edge Extreme', 'Glacier GPS', 'TrailChef Water Bag', 'TrailChef Utensils', 'Star Lite'], ['Hibernator Lite', 'Husky Rope 100', 'Husky Harness Extreme', 'Gran

Apply apriori

In [5]:
tr = TransactionEncoder()
tr_arr = tr.fit(dataset).transform(dataset)
FormatDF = pd.DataFrame(tr_arr, columns=tr.columns_)
FormatDF

,Aloe Relief,Bear Edge,Bear Survival Edge,Blue Steel Max Putter,Blue Steel Putter,BugShield Extreme,BugShield Lotion,BugShield Lotion Lite,BugShield Natural,BugShield Spray,...,TrailChef Canteen,TrailChef Cook Set,TrailChef Cup,TrailChef Deluxe Cook Set,TrailChef Double Flame,TrailChef Kettle,TrailChef Kitchen Kit,TrailChef Single Flame,TrailChef Utensils,TrailChef Water Bag
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5356,False,True,False,False,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
5357,False,False,False,False,False,True,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
5358,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
frequent_itemsets = apriori(FormatDF, min_support = 0.01, use_colnames = True, verbose=1) #0.1, 0.001 barely works w/32gb
frequent_itemsets = frequent_itemsets.sort_values(by=['support'], ascending=False)
frequent_itemsets

Processing 193413 combinations | Sampling itemset size 3


,support,itemsets
11,0.106903,(Canyon Mule Carryall)
102,0.104104,(Sun Shelter 30)
24,0.101119,(Double Edge)
64,0.099067,(Hibernator Lite)
31,0.097948,(Firefly 2)
...,...,...
1790,0.010075,"(Star Dome, Star Gazer 3)"
1061,0.010075,"(Firefly Multi-light, Hibernator Pad)"
1788,0.010075,"(Single Edge, TrailChef Utensils)"
1052,0.010075,"(Firefly Multi-light, Glacier Basic)"


Support (item) = (Transactions relating item) / (Total transactions)
Confidence = (Transactions relating both item1 and item2) / (Total transactions involving item1)

Apriori is traag wanneer grote datasets in memory geladen moeten worden en de min_support klein is, gelukkig is deze dataset niet al te groot